In [ ]:
import sagemaker
import os

In [ ]:
import pathlib as pl

In [ ]:
import datetime

In [ ]:
from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import TensorBoardOutputConfig

In [ ]:
MODEL = "InceptionV3"
EPOCHS = 5
STEPS_PER_EPOCH = 10

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
upload_data = False
if upload_data:
    data_input_path = sess.upload_data('data/severstal-binary')
else:
    data_input_path = "s3://sagemaker-us-east-2-475496805360/data"
print(data_input_path)

In [ ]:
training_input_path = data_input_path + "/train"
validation_input_path = data_input_path + "/validate"

In [ ]:
time_now = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%Mm%Ss")
tensorboard_logs_dir = f"s3://{sess.default_bucket()}/tensorboard_logs_v2/fit/{time_now}_{MODEL}"

In [ ]:
tensorboard_output_config = TensorBoardOutputConfig(s3_output_path=tensorboard_logs_dir)

In [ ]:
tf_estimator = TensorFlow(
    entry_point='fine_tune_eff_net.py', 
    role=role,
    instance_count=1, 
    instance_type='ml.g4dn.xlarge',  # local 'ml.g4dn.xlarge'
    framework_version='2.3', 
    py_version='py37',
    script_mode=True,
    hyperparameters={
        'epochs': EPOCHS,
        "steps-per-epoch": STEPS_PER_EPOCH,
        "log-dir": tensorboard_logs_dir,
        "model": MODEL
    },
    tensorboard_output_config=tensorboard_output_config,
)

In [ ]:
tf_estimator.fit(
    {
        'training': training_input_path, 
        'validation': validation_input_path, 
    }
)